<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
import pandas as pd
import sqlite3
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns


from wordcloud import WordCloud
import numpy as np
import re

from bs4 import BeautifulSoup
import tqdm

import nltk
from itertools import combinations
#from toolz import compose

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')

from nltk.corpus import words
from nltk.tokenize import sent_tokenize
stemmer = SnowballStemmer('english')
from scipy.sparse import coo_matrix, hstack
from tqdm import tqdm

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     /home/krushithreddy0817/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data_with_all_tags = pd.read_csv("data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0


In [3]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [4]:
train.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
1,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
2,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
3,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0
4,'C'-Man,Customs Investigator Cliff Holden (Dean Jagger...,murder,train,imdb,1,1,1421,102,custom investig cliff holden dean jagger retur...,...,0,0,0,0,0,0,0,0,0,0


# CountVectorizer with word and one,bi,tri grams:

In [6]:
vectorize=CountVectorizer(min_df=0.0005,ngram_range=(1,3),analyzer='word')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

In [8]:
y_train= train.drop(['title','plot_synopsis','tags','split','synopsis_source','cnt_dup','tag_count','synopsis_count','synopsis_sent_count','CleanedSynopsis'],axis=1)
y_test= test.drop(['title','plot_synopsis','tags','split','synopsis_source','cnt_dup','tag_count','synopsis_count','synopsis_sent_count','CleanedSynopsis'],axis=1)
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 71)
(2730, 114151)   (2730, 71)


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  1.2min remaining:   16.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  1.3min remaining:    4.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.3min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]


In [10]:
print (grid_search_cv.best_score_)

0.34640132211155866


In [12]:
from sklearn.metrics import classification_report
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4175, Recall: 0.3155, F1-measure: 0.3594
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.16      0.23      0.19       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.00      0.00      0.00        17
 alternate reality       0.33      0.03      0.06        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.10      0.01      0.03        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.00      0.00      0.00        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.16      0.03      0.05       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

In [13]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:  1.8min remaining:   20.8s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:  2.0min remaining:    6.9s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  2.0min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


In [14]:
grid_search_cv.best_score_

0.30238170644900136

In [15]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4141, Recall: 0.2518, F1-measure: 0.3131
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.23      0.08      0.12       108
      adult comedy       0.20      0.04      0.06        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.67      0.24      0.35        17
 alternate reality       0.12      0.07      0.09        29
          anti war       0.29      0.07      0.12        27
       atmospheric       0.05      0.01      0.02        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.20      0.02      0.04        45
    blaxploitation       0.33      0.09      0.14        11
             bleak       0.00      0.00      0.00        39
            boring       0.12      0.03      0.05       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

In [16]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  1.6min remaining:   15.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.8min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


In [17]:
grid_search_cv.best_score_

0.29047834596149885

In [19]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.3594, Recall: 0.2393, F1-measure: 0.2873
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.18      0.10      0.13       108
      adult comedy       0.15      0.11      0.13        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.60      0.18      0.27        17
 alternate reality       0.09      0.10      0.10        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.13      0.04      0.07        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.08      0.02      0.04        45
    blaxploitation       1.00      0.09      0.17        11
             bleak       0.00      0.00      0.00        39
            boring       0.08      0.02      0.03       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

# TfidfVectorizer with word and one,bi,tri grams:

In [20]:
vectorize=TfidfVectorizer(min_df=0.0005,ngram_range=(1,3),analyzer='word')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 71)
(2730, 114151)   (2730, 71)


In [21]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   26.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   46.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  1.2min remaining:   15.4s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  1.2min remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.3min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]


In [22]:
print(grid_search_cv.best_score_)

0.22730778751482614


In [23]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.5854, Recall: 0.1495, F1-measure: 0.2382
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.36      0.05      0.08       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.00      0.00      0.00        17
 alternate reality       0.00      0.00      0.00        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.00      0.00      0.00        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.33      0.02      0.04        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.00      0.00      0.00       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

In [38]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7,10**-8,10**-9],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  2.2min remaining:   20.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.3min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=-1))]


In [39]:
grid_search_cv.best_score_

0.28704854747544173

In [40]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4372, Recall: 0.2226, F1-measure: 0.2950
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.43      0.11      0.18       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       1.00      0.18      0.30        17
 alternate reality       0.25      0.03      0.06        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.30      0.04      0.08        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.17      0.02      0.04        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.18      0.03      0.05       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

In [41]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7,10**-8],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  63 out of  70 | elapsed:  2.2min remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  2.4min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=-1))]
0.2902737479604776


In [42]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4136, Recall: 0.2358, F1-measure: 0.3004
                    precision    recall  f1-score   support

            absurd       0.22      0.04      0.06        54
            action       0.39      0.13      0.19       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       1.00      0.29      0.45        17
 alternate reality       0.20      0.03      0.06        29
          anti war       0.11      0.04      0.06        27
       atmospheric       0.15      0.03      0.05        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.09      0.02      0.04        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.08      0.01      0.02       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

# CountVectorizing with Characters with Tri,Four grams:

In [43]:
vectorize=CountVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 71)
(2730, 93254)   (2730, 71)


In [44]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:  6.7min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:  7.1min remaining:   53.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  7.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  7.6min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.3321515089356638


In [45]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.3549, Recall: 0.3438, F1-measure: 0.3492
                    precision    recall  f1-score   support

            absurd       0.11      0.02      0.03        54
            action       0.15      0.30      0.20       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.00      0.00      0.00        17
 alternate reality       0.22      0.07      0.11        29
          anti war       0.25      0.04      0.06        27
       atmospheric       0.09      0.06      0.07        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.00      0.00      0.00        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.12      0.05      0.07       103
      brainwashing       0.25      0.06      0.10        17
    christian film       0.00      0.00      

In [46]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed: 10.6min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed: 11.9min remaining:   41.1s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed: 11.9min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.2823549432441508


In [47]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.3533, Recall: 0.2369, F1-measure: 0.2836
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.23      0.15      0.18       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.50      0.18      0.26        17
 alternate reality       0.50      0.14      0.22        29
          anti war       1.00      0.04      0.07        27
       atmospheric       0.07      0.01      0.02        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.07      0.02      0.03        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.05      0.09      0.06       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

In [50]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed: 10.0min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 11.0min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.27562701330580375


In [51]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.2623, Recall: 0.2360, F1-measure: 0.2485
                    precision    recall  f1-score   support

            absurd       0.03      0.02      0.02        54
            action       0.20      0.19      0.20       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.09      0.06      0.07        17
 alternate reality       0.19      0.17      0.18        29
          anti war       0.05      0.04      0.04        27
       atmospheric       0.08      0.10      0.09        69
  autobiographical       0.50      0.12      0.20         8
       avant garde       0.10      0.04      0.06        45
    blaxploitation       0.50      0.09      0.15        11
             bleak       0.00      0.00      0.00        39
            boring       0.07      0.24      0.10       103
      brainwashing       0.00      0.00      0.00        17
    christian film       0.00      0.00      

# TfidfVectorizer with Characters with Tri,Four grams:

In [52]:
vectorize=TfidfVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 71)
(2730, 93254)   (2730, 71)


In [53]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:  4.5min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:  4.8min remaining:   36.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  5.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  5.2min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.22100454366817657


In [54]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.5394, Recall: 0.1503, F1-measure: 0.2350
                    precision    recall  f1-score   support

            absurd       1.00      0.02      0.04        54
            action       0.29      0.06      0.11       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.00      0.00      0.00        17
 alternate reality       0.67      0.07      0.12        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.14      0.01      0.03        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.00      0.00      0.00        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.25      0.01      0.02       103
      brainwashing       0.50      0.06      0.11        17
    christian film       0.00      0.00      

In [55]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'),n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:  9.9min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed: 11.2min remaining:   38.8s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed: 11.2min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=-1))]
0.28900520013340236


In [56]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4041, Recall: 0.2354, F1-measure: 0.2975
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.47      0.08      0.14       108
      adult comedy       0.00      0.00      0.00        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.60      0.18      0.27        17
 alternate reality       0.67      0.07      0.12        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.25      0.07      0.11        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.20      0.02      0.04        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.14      0.03      0.05       103
      brainwashing       0.50      0.06      0.11        17
    christian film       0.00      0.00      

In [57]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'),n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed: 10.0min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.6min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=-1))]
0.2868645774551415


In [59]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=y_test.columns))

Precision: 0.4153, Recall: 0.2285, F1-measure: 0.2948
                    precision    recall  f1-score   support

            absurd       0.00      0.00      0.00        54
            action       0.34      0.10      0.16       108
      adult comedy       0.25      0.04      0.06        27
          allegory       0.00      0.00      0.00        22
 alternate history       0.60      0.18      0.27        17
 alternate reality       1.00      0.03      0.07        29
          anti war       0.00      0.00      0.00        27
       atmospheric       0.17      0.03      0.05        69
  autobiographical       0.00      0.00      0.00         8
       avant garde       0.11      0.02      0.04        45
    blaxploitation       0.00      0.00      0.00        11
             bleak       0.00      0.00      0.00        39
            boring       0.20      0.04      0.07       103
      brainwashing       0.50      0.06      0.11        17
    christian film       0.00      0.00      

In [60]:
from prettytable import PrettyTable
tabel = PrettyTable()
tabel.field_names=['Model','Vectorizer','analyzer','ngrams','Precision','recall','f1_score','hyperparameter']
tabel.add_row(['MultinomialNB','CountVectorizer','word','1,3','0.417','0.315','0.359','1'])
tabel.add_row(['SGDClassifier(log)','CountVectorizer','word','1,3','0.414','0.251','0.313','0.01'])
tabel.add_row(['SGDClassifier(hinge)','CountVectorizer','word','1,3','0.359','0.239','0.287','0.01'])
tabel.add_row(['MultinomialNB','TfidfVectorizer','word','1,3','0.585','0.149','0.238','0.01'])
tabel.add_row(['SGDClassifier(log)','TfidfVectorizer','word','1,3','0.437','0.222','0.295','1e-06'])
tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','word','1,3','0.413','0.235','0.300','1e-05'])
tabel.add_row(['MultinomialNB','CountVectorizer','char','3,4','0.354','0.343','0.349','2'])
tabel.add_row(['SGDClassifier(log)','CountVectorizer','char','3,4','0.354','0.236','0.283','0.1'])
tabel.add_row(['SGDClassifier(hinge)','CountVectorizer','char','3,4','0.262','0.236','0.248','0.01'])
tabel.add_row(['MultinomialNB','TfidfVectorizer','char','3,4','0.539','0.150','0.235','0.01'])
tabel.add_row(['SGDClassifier(log)','TfidfVectorizer','char','3,4','0.404','0.235','0.295','1e-06'])
tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','char','3,4','0.415','0.228','0.294','1e-05'])
print(tabel)

+----------------------+-----------------+----------+--------+-----------+--------+----------+----------------+
|        Model         |    Vectorizer   | analyzer | ngrams | Precision | recall | f1_score | hyperparameter |
+----------------------+-----------------+----------+--------+-----------+--------+----------+----------------+
|    MultinomialNB     | CountVectorizer |   word   |  1,3   |   0.417   | 0.315  |  0.359   |       1        |
|  SGDClassifier(log)  | CountVectorizer |   word   |  1,3   |   0.414   | 0.251  |  0.313   |      0.01      |
| SGDClassifier(hinge) | CountVectorizer |   word   |  1,3   |   0.359   | 0.239  |  0.287   |      0.01      |
|    MultinomialNB     | TfidfVectorizer |   word   |  1,3   |   0.585   | 0.149  |  0.238   |      0.01      |
|  SGDClassifier(log)  | TfidfVectorizer |   word   |  1,3   |   0.437   | 0.222  |  0.295   |     1e-06      |
| SGDClassifier(hinge) | TfidfVectorizer |   word   |  1,3   |   0.413   | 0.235  |  0.300   |     1e-05